In [1]:
import pandas as pd
import numpy as np
from urllib.parse import unquote
import os
from contextlib import suppress
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
import pickle  
from sklearn.metrics.pairwise import cosine_similarity

with open('dist/dict_columns_groups.pkl', 'rb') as pkl:
        dict_columns_groups = pickle.load(pkl)
        
with open('dist/lgbm_clf.pkl', 'rb') as pkl:
        LightGBM = pickle.load(pkl)

In [2]:
# Nuestra librería de funciones
from commons import *

In [100]:
data_midi = pd.read_csv('data_midi_drop.csv',sep=",")
data_midi.shape

(88359, 509)

In [101]:
colums_categorized = []
for keys in dict_columns_groups:
    for values in dict_columns_groups[keys]:
        colums_categorized.append(values)
        
colums_categorized

columns_not_categorized = []
for col in data_midi.columns:
    if col not in colums_categorized:
        columns_not_categorized.append(col)

columns_not_categorized.remove('tema')
columns_not_categorized.remove('path')
columns_not_categorized.remove('indice')
columns_not_categorized.remove('Cluster')

In [102]:
def cosine_similarity_row(X__sc, vec_a,indice):
    vec_b = X__sc
    # TODO: vectorizar el cosine_similarity
    cosine_list =cosine_similarity([vec_a], vec_b).reshape(-1)
    similarity = pd.Series(cosine_list,index=indice).sort_values(ascending=False)
    return similarity

In [103]:
data_midi.set_index(data_midi.indice,inplace=True,drop=False)
data_index = data_midi[['indice', 'Cluster']]

data_midi.fillna(0, inplace=True)
X=data_midi.drop(['tema','indice','path', 'Cluster'],axis=1)
data_midi.index

sc = StandardScaler()
sc.fit(X)
X__sc = sc.transform(X)
del data_midi

df_scaled = pd.DataFrame(X__sc,columns=X.columns)
df_scaled.set_index(data_index['indice'],inplace=True)
df_scaled['Cluster'] = data_index['Cluster']
data_index['indice_lower_search'] = data_index['indice'].apply(lambda x: x.lower().replace('-',' ').replace('_',' '))
data_index['indice_lower'] = data_index['indice'].apply(lambda x: x.lower())
del data_index['Cluster']

In [8]:
# Iniciamos nuestra API
import requests
from flask import  Flask, request, jsonify, render_template, send_file, safe_join, abort
app = Flask('recomendar canciones')

In [ ]:
@app.route("/search_titles",methods=['GET'])
def search_titles(data_index=data_index, sc=sc):   
    search=request.args['search']
    page_number =int(request.args['page_number'])
    cant_resultados = int(request.args['cant_resultados'])

    page_from = page_number*cant_resultados
    page_to = page_from + cant_resultados
    print(page_from,page_to)
    result = data_index[data_index['indice_lower_search'].str.contains(search.lower())].indice[page_from:page_to].to_dict()
    response = jsonify(result)
    # Enable Access-Control-Allow-Origin
    response.headers.add("Access-Control-Allow-Origin", "*")
    return response

In [ ]:
@app.route("/similarity_songs",methods=['GET'])
def get_simil_songs():   
    
    duracion_notas =float(request.args['duracion_notas'])
    amplitud_tonal = float(request.args['amplitud_tonal'])
    ritmica_instrument = float(request.args['ritmica_instrument']) 
    ritmica_drums = float(request.args['ritmica_drums'])
    armonia = float(request.args['armonia'])
    dinamica = float(request.args['dinamica'])
    instrumentacion = float(request.args['instrumentacion'])
    tempo = float(request.args['tempo'])
    notas_simultaneas = float(request.args['notas_simultaneas']) 
    duracion_tema = float(request.args['duracion_tema']) 
    others = float(request.args['others'])
    penalizacion = float(request.args['penalizacion'])
    filtrar=request.args['filtrar']
    
    song1 = request.args['song1']
    song2 = request.args['song2']
    
    data = data_midi
        
    data.set_index(data.tema,inplace=True,drop=False)
    data = data.iloc[:,0:len(data_midi.columns)]
    data.fillna(0, inplace=True)
    X=data.drop(['tema','indice','path'],axis=1)
    
    sc = StandardScaler()
    X__sc = sc.fit_transform(X)

    df_scaled = pd.DataFrame(X__sc,columns=X.columns)
    df_scaled.set_index(data.indice,inplace=True)
    df_scaled
    
    mask_category =  np.full((data.shape[0],1), True, dtype=bool)
        
    dict_key_values = {'instrumentacion':instrumentacion,
                     'ritmica_drums':ritmica_drums ,
                     'ritmica_instrument':ritmica_instrument ,
                     'amplitud_tonal':amplitud_tonal ,
                     'dinamica':dinamica, 
                     'duracion_notas1':duracion_notas, 
                     'duracion_notas2':duracion_notas, 
                     'notas_simultaneas':notas_simultaneas ,
                     'tempo':tempo, 
                     'duracion_tema':duracion_tema ,
                     'armonia1':armonia, 
                     'armonia2':armonia, 
                     'armonia3':armonia, 
                     'armonia4':armonia, 
                     'armonia5':armonia}

    
    # La penalizacion genera un orden de magnitud para la distancia minima a la hora de calcular el cosine_similarity
    # Si la penalización es 1, el min_sim_distance sera de 0.1, si es 2 sera de 0.01
    # Como resultante las columnas penalizades tendrán aún menos relevancia a la hora de rastreas similitudes entre temas 
    min_sim_distance = 1 / 10**penalizacion 
    
    for keys in dict_key_values.keys():
        cant_keys = len(dict_columns_groups[keys])
        for column in dict_columns_groups[keys]:
            df_scaled[column] = (df_scaled[column] / np.sqrt(cant_keys)) * (dict_key_values[keys] + min_sim_distance)

    for column in columns_not_categorized:
        df_scaled[column] = (df_scaled[column] / np.sqrt(cant_keys) ) * (others + min_sim_distance)
        
    df_scaled = df_scaled.loc[[song1, song2]]

    if (filtrar == 'True'):
        df_values = df_scaled.sum(axis=0)
        df_values_mask = df_values > 0
        df_scaled = df_scaled.loc[:,df_values_mask]
        print('columnas sin filtrar:',df_values_mask.count())
        
        
    df_diff = df_scaled.diff().abs().iloc[1].T.sort_values(ascending=False)

    
    return df_diff.to_json()

In [ ]:
@app.route("/related_songs",methods=['GET'])
def get_related_songs(df_scaled=df_scaled, sc=sc):   
    data=df_scaled.copy()
    search=request.args['search']
    predict=request.args['predict']
    filtrar=request.args['filtrar']
    duracion_notas =float(request.args['duracion_notas'])
    amplitud_tonal = float(request.args['amplitud_tonal'])
    ritmica_instrument = float(request.args['ritmica_instrument']) 
    ritmica_drums = float(request.args['ritmica_drums'])
    armonia = float(request.args['armonia'])
    dinamica = float(request.args['dinamica'])
    instrumentacion = float(request.args['instrumentacion'])
    tempo = float(request.args['tempo'])
    notas_simultaneas = float(request.args['notas_simultaneas']) 
    duracion_tema = float(request.args['duracion_tema']) 
    cant_resultados = int(request.args['cant_resultados'])
    others = float(request.args['others'])
    penalizacion = float(request.args['penalizacion'])
    page_number =int(request.args['page_number'])
    clusterizar=request.args['clusterizar']
    
    print(search)
    search_song = data_index[data_index['indice_lower'] == search.lower()].indice.iloc[0]
    new_midi = ""
    cluster = ""
    if (search.lower() == 'false'):
        new_midi = get_midi_from_path()   
        tema = new_midi.tema 
        midi_df_cols = pd.DataFrame(columns=data.columns).drop(['Cluster'],axis=1)
        mask_columns = list(set(df_scaled.columns) & set(new_midi.columns))
        new_midi = new_midi[mask_columns]
        midi_df = pd.concat([midi_df_cols, new_midi])
        midi_df.fillna(0, inplace=True)
        new_midi = sc.transform(midi_df) 
        cluster = LightGBM.predict(new_midi)[0]
        print('predicted cluster:',cluster) 
        midi_df = pd.DataFrame(new_midi, columns=midi_df_cols.columns)
        midi_df['Cluster'] = cluster
        data = pd.concat([data, midi_df]) 
        path = '..\GET_FILE\\'         
        data.iloc[data.shape[0] - 1:data.shape[0],:].index = path + tema
        search_song = data.iloc[data.shape[0] - 1:data.shape[0],:].index    
        print('tema:',search_song)
    else:
        cluster = data[data.index == search_song].Cluster.iloc[0]     
        print('cluster:',cluster)   
        
    mask_cluster = data['Cluster'] == cluster
    if (clusterizar.lower() == 'true'):
        data = data[mask_cluster]
        print(cluster, data.shape)
    
    dict_key_values = {'instrumentacion':instrumentacion,
                     'ritmica_drums':ritmica_drums ,
                     'ritmica_instrument':ritmica_instrument ,
                     'amplitud_tonal':amplitud_tonal ,
                     'dinamica':dinamica, 
                     'duracion_notas1':duracion_notas, 
                     'duracion_notas2':duracion_notas, 
                     'notas_simultaneas':notas_simultaneas ,
                     'tempo':tempo, 
                     'duracion_tema':duracion_tema ,
                     'armonia1':armonia, 
                     'armonia2':armonia, 
                     'armonia3':armonia, 
                     'armonia4':armonia, 
                     'armonia5':armonia}

    min_sim_distance = 1 / 10**penalizacion 
    
    for keys in dict_key_values.keys():
        cant_keys = len(dict_columns_groups[keys])
        for column in dict_columns_groups[keys]:
            data[column] = (data[column] / np.sqrt(cant_keys)) * (dict_key_values[keys] + min_sim_distance)

    for column in columns_not_categorized:
        data[column] = (data[column] / np.sqrt(cant_keys) ) * (others + min_sim_distance)
     
    song = data.loc[search_song,:]
    if (isinstance(song, pd.Series) == False):
        song = song.iloc[0]
    
    if (filtrar.lower() == 'true'):
        song_values_mask = (song > 0).index
        data = data.loc[:,song_values_mask]
    print(type(data.loc[search_song,:]))
    
    similarity = cosine_similarity_row(data.to_numpy(), song.array, data.index)
    page_from = page_number*cant_resultados
    page_to = page_from + cant_resultados
    result = similarity.iloc[page_from:page_to:].reset_index()
    result.columns = ['path', 'value']
    result = result.to_dict()
    response = jsonify(result)
    # Enable Access-Control-Allow-Origin
    response.headers.add("Access-Control-Allow-Origin", "*")
    
    return response

In [ ]:
app.config["FILES_PATH"] = "Full_MIDI"
@app.route("/get-file/<path:filename>")
def get_file(filename):
    safe_path = safe_join(app.config["FILES_PATH"], filename)
    print(safe_path)
    try:
        response = send_file(safe_path, as_attachment=True)
        response.headers.add("Access-Control-Allow-Origin", "*")
        return response
    except FileNotFoundError:
        abort(404)

In [ ]:
app.run(host='0.0.0.0')

In [21]:
search='beeth'
page_number =2
cant_resultados = 10

page_from = page_number*cant_resultados
page_to = page_from + cant_resultados
print(page_from,page_to)
result = data_index[data_index['indice_lower_search'].str.contains(search.lower())].indice[page_from:page_to].to_dict()
result

20 30


{"Categories\\Classical\\Beethoven\\Piano Sonatas\\Piano Sonata n08 op13 1mov ''Pathetique''.mid": "Categories\\Classical\\Beethoven\\Piano Sonatas\\Piano Sonata n08 op13 1mov ''Pathetique''.mid",
 "Categories\\Classical\\Beethoven\\Piano Sonatas\\Piano Sonata n08 op13 2mov ''Pathetique''.mid": "Categories\\Classical\\Beethoven\\Piano Sonatas\\Piano Sonata n08 op13 2mov ''Pathetique''.mid",
 "Categories\\Classical\\Beethoven\\Piano Sonatas\\Piano Sonata n08 op13 3mov ''Pathetique''.mid": "Categories\\Classical\\Beethoven\\Piano Sonatas\\Piano Sonata n08 op13 3mov ''Pathetique''.mid",
 "Categories\\Classical\\Beethoven\\Piano Sonatas\\Piano Sonata n14 op27 1mov ''Moonlight''.mid": "Categories\\Classical\\Beethoven\\Piano Sonatas\\Piano Sonata n14 op27 1mov ''Moonlight''.mid",
 "Categories\\Classical\\Beethoven\\Piano Sonatas\\Piano Sonata n14 op27 3mov ''Moonlight''.mid": "Categories\\Classical\\Beethoven\\Piano Sonatas\\Piano Sonata n14 op27 3mov ''Moonlight''.mid",
 "Categories\\Class

In [ ]:
## DROP DUPLICATES CODE

In [ ]:
data_midi['len'] = data_midi.indice.apply( lambda x: len(x))
data_midi.sort_values('len', ascending=False,inplace=True)

In [ ]:
cols = ['scale_note_avg_1','scale_note_avg_2','scale_note_avg_3','scale_note_avg_4','scale_note_avg_5','scale_note_avg_6','scale_note_avg_7']
mask_zeros = data_midi[cols].sum(axis=1) != 0
data_midi_without_zeros = data_midi.loc[mask_zeros,cols]
data_midi_without_zeros_duplicated_mask = data_midi_without_zeros.duplicated(keep='first')
list_index_to_drop = data_midi_without_zeros[data_midi_without_zeros_duplicated_mask].index
len(list_index_to_drop)

In [ ]:
del data_midi['len']

In [ ]:
data_midi_drop_duplicated = data_midi.drop(list_index_to_drop)
data_midi_drop_duplicated.shape

In [ ]:
data_midi_drop_duplicated.to_csv("data_midi_drop.csv", index = False)

In [20]:
with open('columns_not_categorized.pkl', 'wb') as df_pkl:
    pickle.dump(columns_not_categorized, df_pkl)  

In [ ]:
search = 'milonga'
data_index[data_index['indice_lower_search'].str.contains(search.lower())].indice[3:4].to_dict()

In [53]:
df_scaled = df_scaled.astype(np.float16)

In [59]:
df_scaled.to_csv("df_scaled.csv")

TypeError: to_csv() got an unexpected keyword argument 'index_col'

In [21]:
data_index.to_csv("data_index.csv",)

In [60]:
df_scaled = pd.read_csv('df_scaled.csv',sep=",",index_col='indice')

In [77]:
data_index = pd.read_csv('data_index.csv',sep=",",index_col='indice.1')

In [8]:
import os, sys

In [184]:
files_path = str(sys.path[0]) + '\Full_MIDI'

dict = {}
count_files = 0
      
for root, dirs, files in os.walk(files_path, topdown=False):
    for name_file in files:
        dict[count_files] = {"file": os.path.join(root, name_file), "file_c": os.path.join(root, name_file)}
        count_files = count_files + 1

df_files = pd.DataFrame.from_dict(dict, "index")
df_files.sort_values('file',inplace=True)
df_files.reset_index(inplace=True,drop=True)
display(df_files)

,file,file_c
0,C:\dh\FULL_MIDI\Full_MIDI\0\040 Feat Erica Bax...,C:\dh\FULL_MIDI\Full_MIDI\0\040 Feat Erica Bax...
1,C:\dh\FULL_MIDI\Full_MIDI\0\050620B (The Child...,C:\dh\FULL_MIDI\Full_MIDI\0\050620B (The Child...
2,C:\dh\FULL_MIDI\Full_MIDI\0\051230.MID,C:\dh\FULL_MIDI\Full_MIDI\0\051230.MID
3,C:\dh\FULL_MIDI\Full_MIDI\0\060107-Sonnet 73.MID,C:\dh\FULL_MIDI\Full_MIDI\0\060107-Sonnet 73.MID
4,C:\dh\FULL_MIDI\Full_MIDI\0\060222-Tribute to ...,C:\dh\FULL_MIDI\Full_MIDI\0\060222-Tribute to ...
...,...,...
88230,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zyysayr.mid,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zyysayr.mid
88231,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zz_top-shes_just...,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zz_top-shes_just...
88232,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zzdq.MID,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zzdq.MID
88233,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zzgsv.mid,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zzgsv.mid


In [185]:
df_files.file = df_files.file.apply(lambda x: x.replace('C:\dh\FULL_MIDI\Full_MIDI\\','').lower().replace('/', '*'))
df_files

,file,file_c
0,0\040 feat erica baxter - dreams (lost witness...,C:\dh\FULL_MIDI\Full_MIDI\0\040 Feat Erica Bax...
1,0\050620b (the children of the night).mid,C:\dh\FULL_MIDI\Full_MIDI\0\050620B (The Child...
2,0\051230.mid,C:\dh\FULL_MIDI\Full_MIDI\0\051230.MID
3,0\060107-sonnet 73.mid,C:\dh\FULL_MIDI\Full_MIDI\0\060107-Sonnet 73.MID
4,0\060222-tribute to mozart.mid,C:\dh\FULL_MIDI\Full_MIDI\0\060222-Tribute to ...
...,...,...
88230,z\z\zyysayr.mid,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zyysayr.mid
88231,z\z\zz_top-shes_just_killing_me.mid,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zz_top-shes_just...
88232,z\z\zzdq.mid,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zzdq.MID
88233,z\z\zzgsv.mid,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zzgsv.mid


In [186]:
data_index.indice_lower_search = data_index.indice_lower_search.apply(lambda x: x.replace('/', '\\'))

In [187]:
data_index.indice_lower_search.tail(10)

indice
5/5i.mid    5\5i.mid
5/56.mid    5\56.mid
4/4i.mid    4\4i.mid
2/27.mid    2\27.mid
7/7.mid      7\7.mid
9/9.MID      9\9.mid
4/4.mid      4\4.mid
5\5.mid      5\5.mid
2/2.mid      2\2.mid
1/1.MID      1\1.mid
Name: indice_lower_search, dtype: object

In [188]:
df_files_join = df_files.set_index('file').join(data_index.set_index('indice_lower_search'), how='left')

In [190]:
df_delete = df_files_join[df_files_join.indice.isnull()]
df_delete

,file_c,indice,indice_lower


In [182]:
for file in df_delete.file_c.to_list():
    os.remove(file)